<a href="https://colab.research.google.com/github/StillWork/c9/blob/master/gg_67_%EB%8B%A8%EC%96%B4%EB%B2%A1%ED%84%B0_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 단어 임베딩
## 텍스트 분석에서 널리 사용됨
- 자연어 처리, 문서 분류, 감성분석, 저작 식별, QA, 챗봇  

## 토큰화
- 단어, 글자, n-gram

## 인코딩
- 원-핫 인코딩: 단어집 크기의 벡터를 사용한다 (크기가 매우 크고 희소함)
- 토큰 임베딩

## 토큰 임베딩 
- 단어 임베딩
- 글자 임베딩
- n-gram 임베딩

### 원-핫 인코딩은 문장의 구조 정보를 잃어버린다

## 단어 벡터
- 희소 베터대신 밀집 벡터를 사용
- id를 구분하는 용도가 아니라 공간상의 점을 매핑 (거리와 벡터를 제공)

### 생성 방법
- 특정한 문제를 해결하는 과정에서 (모델을 만드는 과정에서) 부수적으로 단어벡터를 만들 수 있다. 단어간의 의미를 반영한다. 의미가 가까우면 거리도 가깝게. 
- 다른 곳에서 만든 단어벡터를 로드하여 사용


### Embedding 계층을 사용하여 쉽게 만들 수 있다
- 정수 인덱스를 벡터로 매핑하는 딕셔너리 구조 (인덱스 크기, 벡터 크기)
- 학습 시키는 데이터에 따라 다른 임베딩이 만들어진다.

### IMDB 영화 리뷰 데이터를 사용한 임베딩 예제



In [1]:
from keras.layers import Flatten, Dense, Embedding
from keras.models import Sequential
import os, os.path
import zipfile
from keras.datasets import imdb
from keras import preprocessing

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:


max_features = 10000
maxlen = 20
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)



In [3]:
x_train.shape

(25000,)

In [4]:
y_train[:3]

array([1, 0, 0])

In [5]:
x_train=preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test=preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

In [6]:
x_train.shape

(25000, 20)

In [7]:
model = Sequential()
model.add(Embedding(10000, 8, input_length=maxlen))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


In [8]:
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 2s 93us/step - loss: 0.6759 - acc: 0.6050 - val_loss: 0.6398 - val_acc: 0.6814
Epoch 2/10
20000/20000 [==============================] - 2s 80us/step - loss: 0.5657 - acc: 0.7427 - val_loss: 0.5467 - val_acc: 0.7206
Epoch 3/10
20000/20000 [==============================] - 2s 82us/step - loss: 0.4752 - acc: 0.7808 - val_loss: 0.5113 - val_acc: 0.7384
Epoch 4/10
20000/20000 [==============================] - 2s 81us/step - loss: 0.4263 - acc: 0.8077 - val_loss: 0.5008 - val_acc: 0.7452
Epoch 5/10
20000/20000 [==============================] - 2s 79us/step - loss: 0.3930 - acc: 0.8258 - val_loss: 0.4981 - val_acc: 0.7538
Epoch 6/10
20000/20000 [==============================] - 2s 80us/step - loss: 0.3668 - acc: 0.8395 - val_loss: 0.5014 - val_acc: 0.7530
Epoch 7/10
20000/20000 [==============================] - 2s 82us/step - loss: 0.3435 - acc: 0.8533 - val_loss: 0.5052 - val_acc:

## 위의 결과는 20개의 단어만 고려한 것임
### 성능이 75% 정도 됨
- 각 단어를 독립적으로 다루었으며, 문장의 구성 정보를 고려하지 않음
- 문장의 구조 정보를 고려하려면 임베딩 층 위에 합성곱이나 순환신경망 층을 추가한다

## 사전훈련된 임베딩 사용
- 사전 학습된 합성곱 신경망을 사용하는 것과 같이 임베딩도 사전훈련된 값을 사용할 수 있다.
### 이 예에서는 성능 개선이 없다 

## 사전 학습 임베딩
- 2013년 구글의 Word2Vect
- 2014년 스탠포드의 GloVe: 단어의 동시출현 통계를 분석 

## 원본 IMDB를 사용하여 (토큰화 되기 전의) 텍스트 처리부터 작업을 하겠다
- 훈련 데이터가 부족할 때 유용하다
- 훈련 데이터가 충분하면 직접 학습하는 것이 낫다

In [9]:
if not os.path.exists("./data"):
    os.makedirs("./data")
if not os.path.exists("./data/IMDB.zip"):
  !wget --no-check-certificate http://mng.bz/0tIo \
  -O ./data/IMDB.zip

--2019-05-03 23:50:15--  http://mng.bz/0tIo
Resolving mng.bz (mng.bz)... 35.166.24.88
Connecting to mng.bz (mng.bz)|35.166.24.88|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://mng.bz/0tIo [following]
--2019-05-03 23:50:16--  https://mng.bz/0tIo
Connecting to mng.bz (mng.bz)|35.166.24.88|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 301 
Location: http://s3.amazonaws.com/text-datasets/aclImdb.zip [following]
--2019-05-03 23:50:17--  http://s3.amazonaws.com/text-datasets/aclImdb.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.98.123
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.98.123|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60711700 (58M) [application/zip]
Saving to: ‘./data/IMDB.zip’

./data/IMDB.zip     100%[===================>]  57.90M  2.22MB/s    in 47s     

2019-05-03 23:51:04 (1.24 MB/s) - ‘./data/IMDB.zip’ sav

In [10]:
!ls data

aclImdb_v1.tar.gz  MoviesampleSubmission.csv  ratings_train.txt
alice_mask.png	   movie_test.tsv	      sampleSubmission.csv
alice.txt	   movie_train.tsv	      stormtrooper_mask.png
a_new_hope.txt	   news			      test.tsv
BEXX0004.txt	   NewsResult.xlsx	      toji
heart.jpg	   News.xlsx		      train.tsv
IMDB.zip	   ratings_test.txt


In [11]:
local_zip = './data/IMDB.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./data')
zip_ref.close()

In [12]:
imdb_dir='./data/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')
labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [13]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100  # We will cut reviews after 100 words
training_samples = 10000  # We will be training on 200 samples
validation_samples = 10000  # We will be validating on 10000 samples
max_words = 10000  # We will only consider the top 10,000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

Found 88582 unique tokens.
Shape of data tensor: (25000, 100)
Shape of label tensor: (25000,)


In [14]:
data=pad_sequences(sequences, maxlen=maxlen)
labels=np.asarray(labels)
data.shape, labels.shape

((25000, 100), (25000,))

In [15]:
if not os.path.exists("./data"):
    os.makedirs("./data")
if not os.path.exists("./data/glove6B.zip"):
  !wget --no-check-certificate http://nlp.stanford.edu/data/glove.6B.zip \
  -O ./data/glove6B.zip

--2019-05-03 23:51:28--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-05-03 23:51:29--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘./data/glove6B.zip’

./data/glove6B.zip  100%[===================>] 822.24M  2.06MB/s    in 8m 17s  

2019-05-03 23:59:46 (1.66 MB/s) - ‘./data/glove6B.zip’ saved [862182613/862182613]



In [16]:
!ls -al data

total 1045976
drwxrwxr-x 6 kjchar kjchar      4096 May  3 23:51 .
drwxrwxr-x 8 kjchar kjchar      4096 May  3 23:59 ..
drwxrwxr-x 4 kjchar kjchar      4096 May  3 23:51 aclImdb
-rw-rw-r-- 1 kjchar kjchar  84125825 Jun 16  2018 aclImdb_v1.tar.gz
-rw-rw-r-- 1 kjchar kjchar      7339 May  3 23:41 alice_mask.png
-rw-rw-r-- 1 kjchar kjchar    148570 May  3 23:40 alice.txt
-rw-rw-r-- 1 kjchar kjchar    326362 May  3 23:45 a_new_hope.txt
-rw-rw-r-- 1 kjchar kjchar    913902 Jan  8 16:24 BEXX0004.txt
-rw-rw-r-- 1 kjchar kjchar 862182613 Oct 25  2015 glove6B.zip
-rw-rw-r-- 1 kjchar kjchar     78856 May  3 23:45 heart.jpg
-rw-rw-r-- 1 kjchar kjchar  60711700 Nov 18  2017 IMDB.zip
drwxr-xr-x 2 kjchar kjchar      4096 May  3 23:31 .ipynb_checkpoints
drwxrwxr-x 3 kjchar kjchar      4096 May  3 23:51 __MACOSX
-rw-rw-r-- 1 kjchar kjchar    596647 Jan  8 15:37 MoviesampleSubmission.csv
-rw-rw-r-- 1 kjchar kjchar   3367149 Jul 13  2018 movie_test.tsv
-rw-rw-r-- 1 kjchar kjchar   8481022 Jul 13  2018 mo

In [17]:
local_zip = './data/glove6B.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./data')
zip_ref.close()

In [18]:
glove_dir='data'
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [19]:
embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [20]:
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_2 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [22]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))
model.save_weights('pre_trained_glove_model.h5')

Train on 10000 samples, validate on 10000 samples
Epoch 1/10
10000/10000 [==============================] - 3s 258us/step - loss: 0.7038 - acc: 0.5559 - val_loss: 0.6551 - val_acc: 0.6479
Epoch 2/10
10000/10000 [==============================] - 2s 223us/step - loss: 0.6141 - acc: 0.6709 - val_loss: 0.6168 - val_acc: 0.6594
Epoch 3/10
10000/10000 [==============================] - 2s 229us/step - loss: 0.5155 - acc: 0.7505 - val_loss: 0.5664 - val_acc: 0.7090
Epoch 4/10
10000/10000 [==============================] - 2s 231us/step - loss: 0.4409 - acc: 0.7899 - val_loss: 0.5868 - val_acc: 0.7021
Epoch 5/10
10000/10000 [==============================] - 2s 226us/step - loss: 0.3791 - acc: 0.8293 - val_loss: 0.6054 - val_acc: 0.7017
Epoch 6/10
10000/10000 [==============================] - 2s 235us/step - loss: 0.3307 - acc: 0.8574 - val_loss: 0.6727 - val_acc: 0.6978
Epoch 7/10
10000/10000 [==============================] - 2s 238us/step - loss: 0.2847 - acc: 0.8823 - val_loss: 0.7464 - 

In [23]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo')
plt.plot(epochs, val_loss, 'b')
plt.title('Training and validation loss')

Text(0.5,1,'Training and validation loss')